# Building a CNN to classify the four shapes from the Kaggle dataset of that name

https://www.kaggle.com/smeschke/four-shapes

In [1]:
import cv2
import numpy as np
import os
# import PIL
import tensorflow as tf

import matplotlib.image as mpimg
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from tensorflow.contrib.layers import fully_connected

/home/cecil/apps/anaconda3/envs/ds/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
def flatten(dimData, images):
    images = np.array(images)
    images = images.reshape(len(images), dimData)
    images = images.astype('float32')
    images /=255
    return images

# Get Data

In [4]:
def get_img_n_lbl(path, folders_arr):
    labels, images = [], []
    key = {}
    for folder in folders_arr:
        print(folder)
        label = folders_arr.index(folder)
        key[folder] = label
        
        for img in os.listdir(path+'/'+folder):
            image = cv2.imread(path+'/'+folder+'/'+img, 0)
            images.append(image)
            labels.append(label)
    print(key)
    return images, labels, key
    

In [5]:
path = os.getcwd()+"/../temp_data_storage"

In [6]:
images, labels, key = get_img_n_lbl(path, ['square','star','triangle','circle'])

square
star
triangle
circle
{'square': 0, 'star': 1, 'triangle': 2, 'circle': 3}


# Split into training and test sets

In [7]:
def split_test_train(images, labels):
    #shuffle data
    labels, images = shuffle(labels, images, random_state=0)    
    
    to_train = 0
    train_images, train_labels, test_images, test_labels = [],[],[],[]
    for img, lbl in zip(images, labels):
        if to_train < 5:
            train_images.append(img)
            train_labels.append(lbl)
            to_train += 1
        else:
            test_images.append(img)
            test_labels.append(lbl)
            to_train = 0
    
            
    return np.asarray(train_images), np.asarray(train_labels), np.asarray(test_images), np.asarray(test_labels)

In [8]:
train_images, train_labels, test_images, test_labels = split_test_train(images, labels)

In [9]:
dataDim = np.prod(images[0].shape)
train_data  = flatten(dataDim, train_images)
test_data = flatten(dataDim, test_images)

In [10]:
#batch_size, height, width, channels = dataset.shape
training_img_cnt, height, width = train_images.shape
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = len(key)


In [14]:
with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")
print(conv1.shape)
print(conv2.shape)

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    print(pool3.shape)
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 50 * 50])
    print(pool3_flat.shape)
    

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")
    print(fc1.shape)
    
    
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    print(logits.shape)
    
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

(?, 200, 200, 32)
(?, 100, 100, 64)
(?, 50, 50, 64)
(?, 160000)
(?, 64)
(?, 4)


In [15]:
n_epochs = 10
batch_size = 200


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
#         print(epoch)
        batch_old = 0

        for iteration in range(training_img_cnt // batch_size):
            X_batch = train_data[batch_old: batch_old + batch_size]
            y_batch = train_labels[batch_old: batch_old + batch_size]
#             print(type(y_batch))
#             print(iteration)s
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            batch_old += batch_size
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
#     acc_test = accuracy.eval(feed_dict={X: test_data, y: test_labels})
#     print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
        acc_test = accuracy.eval(feed_dict={X: test_data[:200], y: test_labels[:200]})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./models/four_shape.ckpt")
    

0 Train accuracy: 0.995 Test accuracy: 0.99
1 Train accuracy: 1.0 Test accuracy: 1.0
2 Train accuracy: 1.0 Test accuracy: 1.0
3 Train accuracy: 1.0 Test accuracy: 1.0
4 Train accuracy: 1.0 Test accuracy: 1.0
5 Train accuracy: 1.0 Test accuracy: 1.0
6 Train accuracy: 1.0 Test accuracy: 1.0
7 Train accuracy: 1.0 Test accuracy: 1.0
8 Train accuracy: 1.0 Test accuracy: 1.0
9 Train accuracy: 1.0 Test accuracy: 1.0


In [13]:
reset_graph()

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./models/four_shape.ckpt")
    init.run()
    correct.eval(feed_dict={X: test_data[:5]})